In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.options.display.max_rows = 10

In [3]:
# Q1 : Dans quel arrondissement il y a le plus de places disponibles
# aux plages de stationnement ouverts?
# Q2 : Quels types des stationnements interdits étaient plus / moins populaires
# dans arrondissements de Paris ?
# Q3 : Dans quels mois était plus / moins garé plus longtemps / plus courts ?

In [4]:
# Afficher les colonnes
df = pd.read_csv('stationnements.txt', sep="\t",header=0)
# print(df.head())
df.columns

Index(['identifiant', 'Regime_prioritaire', 'Regime_particulier',
       'Arrondissement', 'Zone_Residentielle', 'Tarification', 'Type_de_voie',
       'Nom_de_la_voie', 'Parite', 'Longueur', 'Longueur_calculee',
       'Signalisation_horizontale', 'Signalisation_verticale',
       'Conformite_signalisation', 'Plage_horaire_1-Debut',
       'Plage_horaire_1-Fin', 'Plage_horaire_2-Debut', 'Plage_horaire_2-Fin',
       'Date_du_releve', 'Derniere_date_edition', 'Code_Voie_Ville_de_Paris',
       'Numero_Sequentiel_Tronçon_Voie', 'Zone_ASP',
       'Numero_Section_Territoriale_de_Voirie', 'geo_point_2d'],
      dtype='object')

In [5]:
# On va tirer les colonnes concernées
liste_colonnes = [ 
            "identifiant", 
            "Arrondissement", 
            "Regime_particulier",
            "Plage_horaire_1-Debut", 
            "Plage_horaire_1-Fin", 
            "Plage_horaire_2-Debut", 
            "Plage_horaire_2-Fin",
            "Date_du_releve",
            "Derniere_date_edition",
            "geo_point_2d",]
df_new = df.loc[:, liste_colonnes]
df_new.head()

,identifiant,Arrondissement,Regime_particulier,Plage_horaire_1-Debut,Plage_horaire_1-Fin,Plage_horaire_2-Debut,Plage_horaire_2-Fin,Date_du_releve,Derniere_date_edition,geo_point_2d
0,3210,13.0,Livraison_BUS,19:30,07:30,09:30,16:30,2020-08-31,2017-08-07,"48.8369632816,2.34941389"
1,3232,13.0,Stationnement_genant,NaN,NaN,NaN,NaN,2020-07-21,2020-08-13,"48.8215395201,2.37281426625"
2,3271,5.0,Arret_vigipirate_non_perennise,NaN,NaN,NaN,NaN,2020-08-31,2017-08-08,"48.8484963453,2.34436146098"
3,3280,5.0,Arret_vigipirate_perennise,NaN,NaN,NaN,NaN,2020-08-31,2017-10-26,"48.8472007904,2.34461920102"
4,3277,5.0,Arret_genant_divers,NaN,NaN,NaN,NaN,2020-08-31,2017-11-22,"48.8402284901,2.33680230973"


In [6]:
# Naviger les valeurs des colonnes pour calculer la durée
liste_colonnes_q1 = [
                "identifiant", 
                "Arrondissement", 
                "Plage_horaire_1-Debut", 
                "Plage_horaire_1-Fin", 
                "Plage_horaire_2-Debut", 
                "Plage_horaire_2-Fin"
                ]
for col in liste_colonnes_q1:
    print("{} : {}".format(col, df_new[col].unique()))

identifiant : ['3210' '3232' '3271' ... '10812' '10848' '10859']
Arrondissement : [13.  5.  6. 11. 20. 18. 19. 15. 12.  4.  7. 16. 14.  9. 17. nan  2.  8.
 10. 21.  1. 22.  3. -1.]
Plage_horaire_1-Debut : ['19:30' nan '09:30' '05:00' '07:00' '19:00' '07:30' '14:00' '19:20'
 '08:15']
Plage_horaire_1-Fin : ['07:30' nan '16:30' '19:00' '14:30' '18:00' '07:00' '09:30' '17:30'
 '07:20' '20:00' '08:45']
Plage_horaire_2-Debut : ['09:30' nan '19:30' '07:30' '05:00' '16:30' '20:00' '19:00' '09:38'
 '09:00' '16:15']
Plage_horaire_2-Fin : ['16:30' nan '07:30' '14:30' '09:30' '19:30' '22:00' '18:30' '16:00'
 '16:29' '16:45' '17:30']


In [7]:
# Afficher les colonnes qui contiennent Nan
for col in liste_colonnes_q1:
    print("{} : {}".format(col, df_new[col].isnull().any()))

identifiant : False
Arrondissement : True
Plage_horaire_1-Debut : True
Plage_horaire_1-Fin : True
Plage_horaire_2-Debut : True
Plage_horaire_2-Fin : True


In [8]:
# Toutes les colonnes 'plage horaire' ont le Nan,
# c'est-à-dire qu'il y a des places ont tous Nan dans les 'plage horaire'
# On compte 3 types de créneaux : 
#    A,Plage horaire 1; 
#    B,Plage horaire 2; 
#    C, 24h pour les place avec que Nan dans 'plage horaire'

In [9]:
# On va transférer la donné 'plage horaire' en minutes pour faciliter les calculs

In [10]:
# Remplacer Nan par '*'
df_new = df_new.fillna(value=0)

for col in liste_colonnes_q1:
    print("{} : {}".format(col, df_new[col].isnull().any()))

identifiant : False
Arrondissement : False
Plage_horaire_1-Debut : False
Plage_horaire_1-Fin : False
Plage_horaire_2-Debut : False
Plage_horaire_2-Fin : False


In [11]:
# Changement des noms colonnes
df_new = df_new.rename(columns={
                            "Plage_horaire_1-Debut":"P1D", 
                            "Plage_horaire_1-Fin":"P1F",
                            "Plage_horaire_2-Debut":"P2D",
                            "Plage_horaire_2-Fin":"P2F",
                        }
                      )
df_new.columns

Index(['identifiant', 'Arrondissement', 'Regime_particulier', 'P1D', 'P1F',
       'P2D', 'P2F', 'Date_du_releve', 'Derniere_date_edition',
       'geo_point_2d'],
      dtype='object')

In [12]:
# 'plage horaire' en minutes
liste_PH = ['P1D', 'P1F', 'P2D', 'P2F']

for row in range(df_new.shape[0]):
    tmp = 0
    for col in liste_PH:
        if df_new[col][row] != 0:
            tmp = int(df_new[col][row][:2])*60 + int(df_new[col][row][-2:])            
            df_new[col][row] = tmp
        else:       
            pass
print('finit')
df_new.head()

c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


finit


,identifiant,Arrondissement,Regime_particulier,P1D,P1F,P2D,P2F,Date_du_releve,Derniere_date_edition,geo_point_2d
0,3210,13.0,Livraison_BUS,1170,450,570,990,2020-08-31,2017-08-07,"48.8369632816,2.34941389"
1,3232,13.0,Stationnement_genant,0,0,0,0,2020-07-21,2020-08-13,"48.8215395201,2.37281426625"
2,3271,5.0,Arret_vigipirate_non_perennise,0,0,0,0,2020-08-31,2017-08-08,"48.8484963453,2.34436146098"
3,3280,5.0,Arret_vigipirate_perennise,0,0,0,0,2020-08-31,2017-10-26,"48.8472007904,2.34461920102"
4,3277,5.0,Arret_genant_divers,0,0,0,0,2020-08-31,2017-11-22,"48.8402284901,2.33680230973"


In [13]:
# Naviger les valeurs des 'plage horaire'
for col in liste_PH:
    print("{} : {}".format(col, df_new[col].unique()))

P1D : [1170 0 570 300 420 1140 450 840 1160 495]
P1F : [450 0 990 1140 870 1080 420 570 1050 440 1200 525]
P2D : [570 0 1170 450 300 990 1200 1140 578 540 975]
P2F : [990 0 450 870 570 1170 1320 1110 960 989 1005 1050]


In [14]:
# Calculer les durées des plages 
# Créer la colonne Duree et la remplir
df_new['Duree'] = pd.Series([])

for row in range(df_new.shape[0]):
    # Plage Horaire 1
    créneau_1 = 0
    if df_new['P1F'][row] == 0:
        pass
    elif df_new['P1F'][row] > df_new['P1D'][row]:
        créneau_1 = df_new['P1F'][row] - df_new['P1D'][row]
    else:
        créneau_1 = 24*60 + (df_new['P1F'][row] - df_new['P1D'][row])

    # Plage Horaire 2
    créneau_2 = 0
    if df_new['P2F'][row] == 0:
        pass
    elif df_new['P2F'][row] > df_new['P2D'][row]:
        créneau_2 = df_new['P2F'][row] - df_new['P2D'][row]
    else:
        créneau_2 = 24*60 + (df_new['P2F'][row] - df_new['P2D'][row])
        
    # La colonne 'Duree'
    if (créneau_1 + créneau_2) == 0:
        df_new['Duree'][row] = 24*60
    else:
        df_new['Duree'][row] = créneau_1 + créneau_2

print('finit')
print(df_new['Duree'].value_counts(dropna=False))

c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

finit
1440.0    19760
1140.0      683
1110.0       10
1170.0        6
1740.0        6
          ...  
1139.0        1
1380.0        1
1132.0        1
210.0         1
660.0         1
Name: Duree, Length: 21, dtype: int64


In [15]:
# Naviger la Dataframe
df_new.head()

,identifiant,Arrondissement,Regime_particulier,P1D,P1F,P2D,P2F,Date_du_releve,Derniere_date_edition,geo_point_2d,Duree
0,3210,13.0,Livraison_BUS,1170,450,570,990,2020-08-31,2017-08-07,"48.8369632816,2.34941389",1140.0
1,3232,13.0,Stationnement_genant,0,0,0,0,2020-07-21,2020-08-13,"48.8215395201,2.37281426625",1440.0
2,3271,5.0,Arret_vigipirate_non_perennise,0,0,0,0,2020-08-31,2017-08-08,"48.8484963453,2.34436146098",1440.0
3,3280,5.0,Arret_vigipirate_perennise,0,0,0,0,2020-08-31,2017-10-26,"48.8472007904,2.34461920102",1440.0
4,3277,5.0,Arret_genant_divers,0,0,0,0,2020-08-31,2017-11-22,"48.8402284901,2.33680230973",1440.0


In [16]:
# Extrait 'mois'
# Explorer les colonnes 'Date_du_releve' et 'Derniere_date_edition'
print("{} : \n{}".format('Date_du_releve', df_new['Date_du_releve'].value_counts(normalize=True)))
print("{} : \n{}".format('Derniere_date_edition', df_new['Derniere_date_edition'].value_counts(normalize=True)))

Date_du_releve : 
2020-08-31    0.703254
2018-05-01    0.013854
2020-08-01    0.013074
2020-07-07    0.010049
2020-07-09    0.009610
                ...   
1995-11-01    0.000049
2021-07-01    0.000049
2020-05-01    0.000049
2020-09-30    0.000049
2019-07-12    0.000049
Name: Date_du_releve, Length: 126, dtype: float64
Derniere_date_edition : 
2020-08-24    0.017220
2020-08-14    0.017172
2020-08-20    0.016537
2017-08-17    0.015903
2020-08-28    0.014537
                ...   
2020-01-06    0.000049
2021-03-25    0.000049
2018-12-19    0.000049
2018-12-06    0.000049
2019-08-06    0.000049
Name: Derniere_date_edition, Length: 397, dtype: float64


In [17]:
# Il y 70% de lignes ont '2020-08-31' dans 'Date_du_releve' qui va géner le resultat.
# On va donc prendre la colonne 'Date_du_releve'

In [18]:
# Transférer jour à mois et ajouter la colonne 'Mois'
df_new['Mois'] = pd.Series([])
for row in range(df_new.shape[0]):
    df_new['Mois'][row] = df_new['Date_du_releve'][row][5:7]
print('finit')
df_new.head()

c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


finit


,identifiant,Arrondissement,Regime_particulier,P1D,P1F,P2D,P2F,Date_du_releve,Derniere_date_edition,geo_point_2d,Duree,Mois
0,3210,13.0,Livraison_BUS,1170,450,570,990,2020-08-31,2017-08-07,"48.8369632816,2.34941389",1140.0,8.0
1,3232,13.0,Stationnement_genant,0,0,0,0,2020-07-21,2020-08-13,"48.8215395201,2.37281426625",1440.0,7.0
2,3271,5.0,Arret_vigipirate_non_perennise,0,0,0,0,2020-08-31,2017-08-08,"48.8484963453,2.34436146098",1440.0,8.0
3,3280,5.0,Arret_vigipirate_perennise,0,0,0,0,2020-08-31,2017-10-26,"48.8472007904,2.34461920102",1440.0,8.0
4,3277,5.0,Arret_genant_divers,0,0,0,0,2020-08-31,2017-11-22,"48.8402284901,2.33680230973",1440.0,8.0


In [19]:
# Séparer 'geo_point_2d' en 2 colonnes 'Latitude' et 'Longitude'
df_geo = df_new['geo_point_2d'].str.split(pat=',', expand=True).rename(columns={0:'Latitude', 1:"Longitude"})
df_geo

,Latitude,Longitude
0,48.8369632816,2.34941389
1,48.8215395201,2.37281426625
2,48.8484963453,2.34436146098
3,48.8472007904,2.34461920102
4,48.8402284901,2.33680230973
...,...,...
20494,48.843261749,2.32607288492
20495,48.8529651867,2.38812681227
20496,48.851261897,2.39282500813
20497,48.8449893867,2.38869361782


In [20]:
df_new = df_new.join(df_geo, how='right')
df_new.head()

,identifiant,Arrondissement,Regime_particulier,P1D,P1F,P2D,P2F,Date_du_releve,Derniere_date_edition,geo_point_2d,Duree,Mois,Latitude,Longitude
0,3210,13.0,Livraison_BUS,1170,450,570,990,2020-08-31,2017-08-07,"48.8369632816,2.34941389",1140.0,8.0,48.8369632816,2.34941389
1,3232,13.0,Stationnement_genant,0,0,0,0,2020-07-21,2020-08-13,"48.8215395201,2.37281426625",1440.0,7.0,48.8215395201,2.37281426625
2,3271,5.0,Arret_vigipirate_non_perennise,0,0,0,0,2020-08-31,2017-08-08,"48.8484963453,2.34436146098",1440.0,8.0,48.8484963453,2.34436146098
3,3280,5.0,Arret_vigipirate_perennise,0,0,0,0,2020-08-31,2017-10-26,"48.8472007904,2.34461920102",1440.0,8.0,48.8472007904,2.34461920102
4,3277,5.0,Arret_genant_divers,0,0,0,0,2020-08-31,2017-11-22,"48.8402284901,2.33680230973",1440.0,8.0,48.8402284901,2.33680230973


In [23]:
# Créer une dataframe clean qui est prêt pour calculer les résultats
liste_colonnes_clean = [
            "identifiant"         ,
            "Arrondissement"      ,
            "Regime_particulier"  ,
            "Duree"               ,
            "Mois"                ,
            "Latitude"            ,
            "Longitude"           ,
            ]
df_clean = df_new.loc[:, liste_colonnes_clean]
print(df_clean.head())
df_clean.info()

  identifiant  Arrondissement              Regime_particulier   Duree  Mois  \
0        3210            13.0                   Livraison_BUS  1140.0   8.0   
1        3232            13.0            Stationnement_genant  1440.0   7.0   
2        3271             5.0  Arret_vigipirate_non_perennise  1440.0   8.0   
3        3280             5.0      Arret_vigipirate_perennise  1440.0   8.0   
4        3277             5.0             Arret_genant_divers  1440.0   8.0   

        Latitude      Longitude  
0  48.8369632816     2.34941389  
1  48.8215395201  2.37281426625  
2  48.8484963453  2.34436146098  
3  48.8472007904  2.34461920102  
4  48.8402284901  2.33680230973  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20499 entries, 0 to 20498
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   identifiant         20499 non-null  object 
 1   Arrondissement      20499 non-null  float64
 2   Regime_parti

In [24]:
# Sauvegarder la dataframe to un fichier csv
df_clean.to_csv('df_clean.csv', index=False)